In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 

In [2]:
target_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Cherry_(including_sour)___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [5]:
# Load the images and labels
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        X.append(img_array)
        y.append(target_labels.index(label))

In [6]:
from tensorflow.keras.utils import to_categorical

In [7]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [8]:
X.shape

(60930, 128, 128, 3)

In [9]:
y.shape

(60930, 33)

In [10]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
X_train.shape

(48744, 128, 128, 3)

In [12]:
X_test.shape

(12186, 128, 128, 3)

In [13]:
y_train.shape

(48744, 33)

In [14]:
y_test.shape

(12186, 33)

In [15]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [16]:
len(target_labels)

33

In [17]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-15 19:26:47.087520: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1524/1524 [==============================] - 280s 183ms/step - loss: 1.3948 - accuracy: 0.5772 - val_loss: 0.8999 - val_accuracy: 0.7108
Epoch 2/10
1524/1524 [==============================] - 297s 195ms/step - loss: 0.6165 - accuracy: 0.8009 - val_loss: 0.7341 - val_accuracy: 0.7652
Epoch 3/10
1524/1524 [==============================] - 349s 229ms/step - loss: 0.4266 - accuracy: 0.8597 - val_loss: 0.2650 - val_accuracy: 0.9108
Epoch 4/10
1524/1524 [==============================] - 416s 273ms/step - loss: 0.3334 - accuracy: 0.8905 - val_loss: 0.2144 - val_accuracy: 0.9271
Epoch 5/10
1524/1524 [==============================] - 431s 283ms/step - loss: 0.2789 - accuracy: 0.9080 - val_loss: 0.2591 - val_accuracy: 0.9160
Epoch 6/10
1524/1524 [==============================] - 433s 284ms/step - loss: 0.2338 - accuracy: 0.9221 - val_loss: 0.2256 - val_accuracy: 0.9232
Epoch 7/10
1524/1524 [==============================] - 438s 287ms/step - loss: 0.2123 - accuracy: 0.9291 - val_loss: 0.312

In [19]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 95.31%


In [20]:
cnn_scores

[0.14225374162197113, 0.9531429409980774]

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("CNN Accuracy: %.2f%%" % (accuracy * 100))
print("CNN Precision: %.2f%%" % (precision * 100))
print("CNN Recall: %.2f%%" % (recall * 100))
print("CNN F1 score: %.2f%%" % (f1 * 100))

381/381 [==============================] - 30s 77ms/step
CNN Accuracy: 95.31%
CNN Precision: 95.50%
CNN Recall: 95.31%
CNN F1 score: 95.31%


# RF

In [18]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

2023-05-17 21:44:58.508728: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1524/1524 [==============================] - 79s 51ms/step


In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
# Train a Random Forest classifier on the extracted features
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

RandomForestClassifier(random_state=42)

In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)

# Calculate and print the Accuracy
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f%%" % (rf_precision * 100))

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f%%" % (rf_recall * 100))

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f%%" % (rf_f1_score * 100))

381/381 [==============================] - 18s 48ms/step
Random Forest Classifier Accuracy: 29.29%
Random Forest Classifier Precision: 27.90%
Random Forest Classifier Recall: 29.29%
Random Forest Classifier F1 Score: 27.60%


# Tes

In [26]:
#Label assignment
label=['Apple___Apple_scab', 
       'Apple___Black_rot', 
       'Apple___Cedar_apple_rust', 
       'Apple___healthy', 
       'Cherry_(including_sour)___healthy', 
       'Cherry_(including_sour)___Powdery_mildew', 
       'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 
       'Corn_(maize)___Common_rust_', 
       'Corn_(maize)___healthy', 
       'Corn_(maize)___Northern_Leaf_Blight', 
       'Grape___Black_rot', 
       'Grape___Esca_(Black_Measles)', 
       'Grape___healthy', 
       'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 
       'Peach___Bacterial_spot', 
       'Peach___healthy', 
       'Pepper,_bell___Bacterial_spot', 
       'Pepper,_bell___healthy', 
       'Potato___Early_blight', 
       'Potato___healthy', 
       'Potato___Late_blight', 
       'Strawberry___healthy', 
       'Strawberry___Leaf_scorch', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [27]:
from tensorflow.keras.preprocessing import image

In [41]:
#path=input("Enter your image path-: ")
def testing(path):
    test_image=image.load_img(path,target_size=(128,128))
    #print(test_image)
    test_image=image.img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    result = cnn_model.predict(test_image)

    #print(f"Result is --> {result}")
    fresult=np.max(result)
    label2=label[result.argmax()]
    print(f"Result is --> {result}")
    print(f"FResult is --> {fresult}")
    print(f"your leaf disease is --> {label2}")
#testing(input("Enter your image path-: "))
path='tes-1.JPG'
testing(path)

1/1 [==============================] - 0s 15ms/step
Result is --> [[2.63115908e-07 1.07622019e-03 9.98918891e-01 2.66988498e-13
  1.12537402e-09 2.00369304e-11 9.16327192e-09 2.05335660e-09
  3.44800891e-18 2.81340793e-12 3.10764504e-11 2.74405111e-15
  2.06363548e-09 1.84235926e-16 6.35832365e-09 2.18337457e-12
  7.26870269e-07 1.24834060e-07 1.25660836e-11 7.42666106e-09
  1.22567234e-14 2.05018904e-14 6.82602100e-11 4.36812010e-11
  1.59527270e-11 2.66565806e-12 5.44153472e-11 1.56892038e-15
  3.74633646e-06 1.80190593e-17 2.66315170e-09 1.33182285e-11
  2.62267340e-13]]
FResult is --> 0.998918890953064
your leaf disease is --> Apple___Cedar_apple_rust


In [49]:
import os

def testing_folder_cnn(path):
    test_image = image.load_img(path, target_size=(128, 128))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = cnn_model.predict(test_image)

    fresult = np.max(result)
    label2 = label[result.argmax()]
    return fresult, label2

folder_path = 'data_penyakit/images_for_test'

results = []
for filename in os.listdir(folder_path):
    image_path = os.path.join(folder_path, filename)
    print(f"Testing image: {filename}")
    fresult, label2 = testing_folder_cnn(image_path)
    print(f"FResult is --> {fresult}")
    print(f"Your leaf disease is --> {label2}")
    print("\n")
    
    results.append({
        'filename': filename,
        'fresult': fresult,
        'label': label2
    })

df_cnn = pd.DataFrame(results)
df_cnn.to_csv('results-cnn.csv', index=False)

Testing image: AppleScab3.JPG
1/1 [==============================] - 0s 18ms/step
FResult is --> 0.8601953387260437
Your leaf disease is --> Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot


Testing image: TomatoEarlyBlight3.JPG
1/1 [==============================] - 0s 14ms/step
FResult is --> 0.8626644611358643
Your leaf disease is --> Tomato___Early_blight


Testing image: PotatoHealthy1.JPG
1/1 [==============================] - 0s 13ms/step
FResult is --> 0.9657799005508423
Your leaf disease is --> Potato___healthy


Testing image: AppleScab2.JPG
1/1 [==============================] - 0s 14ms/step
FResult is --> 0.9971644282341003
Your leaf disease is --> Apple___Apple_scab


Testing image: TomatoEarlyBlight1.JPG
1/1 [==============================] - 0s 12ms/step
FResult is --> 0.6627451181411743
Your leaf disease is --> Tomato___Late_blight


Testing image: PotatoHealthy2.JPG
1/1 [==============================] - 0s 14ms/step
FResult is --> 0.998801589012146
Your leaf diseas

In [42]:
def testing_rf(path, rf_classifier):
    test_image = image.load_img(path, target_size=(128, 128))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    rf_features = cnn_model.predict(test_image)
    rf_features = rf_features.reshape(rf_features.shape[0], -1)
    result = rf_classifier.predict(rf_features)

    label2 = label[result[0]]
    print(f"Result is --> {result}")
    print(f"Your leaf disease is --> {label2}")

# testing(input("Enter your image path-: "))
path = 'tes-1.JPG'
testing_rf(path, rf_classifier)

1/1 [==============================] - 0s 15ms/step
Result is --> [2]
Your leaf disease is --> Apple___Cedar_apple_rust


In [50]:
import os

def testing_folder_rf(path, rf_classifier):
    test_image = image.load_img(path, target_size=(128, 128))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)

    rf_features = cnn_model.predict(test_image)
    rf_features = rf_features.reshape(rf_features.shape[0], -1)
    result = rf_classifier.predict(rf_features)

    label2 = label[result[0]]
    return label2

folder_path = 'data_penyakit/images_for_test'
results = []

for filename in os.listdir(folder_path):
    image_path = os.path.join(folder_path, filename)
    print(f"Testing image: {filename}")
    label2 = testing_folder_rf(image_path, rf_classifier)
    print(f"Your leaf disease is --> {label2}")
    print("\n")

    results.append({
        'filename': filename,
        'label': label2
    })

df_rf = pd.DataFrame(results)

# Save the dataframe into a CSV file
df_rf.to_csv('results-rf.csv', index=False)

Testing image: AppleScab3.JPG
1/1 [==============================] - 0s 17ms/step
Your leaf disease is --> Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot


Testing image: TomatoEarlyBlight3.JPG
1/1 [==============================] - 0s 13ms/step
Your leaf disease is --> Tomato___Early_blight


Testing image: PotatoHealthy1.JPG
1/1 [==============================] - 0s 12ms/step
Your leaf disease is --> Potato___healthy


Testing image: AppleScab2.JPG
1/1 [==============================] - 0s 13ms/step
Your leaf disease is --> Apple___Apple_scab


Testing image: TomatoEarlyBlight1.JPG
1/1 [==============================] - 0s 12ms/step
Your leaf disease is --> Tomato___Late_blight


Testing image: PotatoHealthy2.JPG
1/1 [==============================] - 0s 11ms/step
Your leaf disease is --> Potato___healthy


Testing image: AppleScab1.JPG
1/1 [==============================] - 0s 23ms/step
Your leaf disease is --> Apple___Apple_scab


Testing image: TomatoEarlyBlight4.JPG
1/1 [====